# Test GPU

In [ ]:
import torch
torch.cuda.current_device()
torch.cuda.get_device_name(0)

In [ ]:
! nvidia-smi

# Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""
Folder => "Alzheimer Competition"
https://drive.google.com/drive/folders/1-NIobsSrpU5JyUfu0bqbuWZU32GVZ7-m?usp=sharing

Folder => "SayeedColab"
https://drive.google.com/drive/folders/1wnL3y-kltnGwsV9dhha6BbOI7UwF0YJW?usp=sharing

"""

# Copy Files

In [ ]:
import shutil
import os
import glob

shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/sampler.py", "sampler.py")
shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/ranger.py", "ranger.py")
shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/resnext.py", "resnext.py")
shutil.copyfile("/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/resnet.py", "resnet.py")

!jar -xf "/content/drive/My Drive/AlzheimerStallCatcher3DConvPointCloud/traintestlist.zip";

print("Done")

Done


In [ ]:
!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/micro_1.zip";
print("partition 1 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/micro_2.zip";
print("partition 2 imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/micro_3.zip";
print("partition 3 imported")

files = [f for f in glob.glob("micro/" + "*" + ".pt", recursive=True)]
print("Total: " + str(len(files)) + " should be 2399")

partition 1 imported
partition 2 imported
partition 3 imported
Total: 2399 should be 2399


In [ ]:
# NEW DATA
"""
!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/stall.zip";
print("new data stall imported")

!jar -xf "/content/drive/My Drive/SayeedColab/Alzheimer Data/nonstall_0.zip";
print("new data nonstall_0 imported")
"""

In [ ]:
! rm -rf loss_acc_curve # remove any folder fully

# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
import time

from sklearn.metrics import matthews_corrcoef as mcc

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.utils.data
from torchvision.models.video import r3d_18

torch.manual_seed(100)

import csv

from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import itertools

# Dataset

### Balanced Batch Sampler


In [ ]:
import torchvision
import torch.utils.data
import random


class BalancedBatchSampler(torch.utils.data.sampler.Sampler):
    def __init__(self, dataset):
        self.dataset = {}
        self.balanced_max = 0
        # Save all the indices for all the classes
        for idx in range(0, len(dataset)):
            label = self._get_label(dataset, idx)
            if label not in self.dataset:
                self.dataset[label] = []
            self.dataset[label].append(idx)
            self.balanced_max = len(self.dataset[label]) \
                if len(self.dataset[label]) > self.balanced_max else self.balanced_max
        
        # Oversample the classes with fewer elements than the max
        for label in self.dataset:
            while len(self.dataset[label]) < self.balanced_max:
                self.dataset[label].append(random.choice(self.dataset[label]))
    
        self.keys = list(self.dataset.keys())
        self.currentkey = 0

    def __iter__(self):
        while len(self.dataset[self.keys[self.currentkey]]) > 0:
            yield self.dataset[self.keys[self.currentkey]].pop()
            self.currentkey = (self.currentkey + 1) % len(self.keys)

    
    def _get_label(self, dataset, idx):
        dataset_type = type(dataset)
        if dataset_type is torchvision.datasets.MNIST:
            return dataset.train_labels[idx].item()
        elif dataset_type is torchvision.datasets.ImageFolder:
            return dataset.imgs[idx][1]
        else:
            (image_sequence, target) = dataset.__getitem__(idx)
            return target

    def __len__(self):
        return self.balanced_max*len(self.keys)

### Custom Dataset Class

In [ ]:
from torch.utils.data import Dataset, DataLoader

class VoxelDataset(Dataset):
    def __init__(self, dataset_path, split_path, split_number, training):
        self.training = training
        self.sequences, self.labels = self._extract_sequence_paths_and_labels(dataset_path, split_path, split_number,
                                                                              training)  # creating a list of directories where the extracted frames are saved
        self.label_names = ["Non-stalled", "Stalled"]  # Getting the label names or name of the class
        self.num_classes = len(self.label_names)  # Getting the number of class


    def _extract_sequence_paths_and_labels(
            self, dataset_path, split_path="data/traintestlist", split_number=0, training=True
    ):
        """ Extracts paths to sequences given the specified train / test split """
        fn = f"fold_{split_number}_train.csv" if training else f"fold_{split_number}_test.csv"
        split_path = os.path.join(split_path, fn)
        df = pd.read_csv(split_path)
        file_name = df['filename'].values
        all_labels = df['class'].values
        sequence_paths = []
        classes = []
        for i, video_name in enumerate(file_name):
            seq_name = video_name.split(".mp4")[0]
            sequence_paths += [os.path.join(dataset_path, seq_name).replace('\\', '/')]
            classes += [all_labels[i]]
        return sequence_paths, classes

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, index):
        sequence_path = self.sequences[index % len(self)]
        target = self.labels[index % len(self)]

        X = torch.load(sequence_path + ".pt")

        return X, target


### Create Dataset

In [ ]:
batch_size = 128

split_number = 0

augmentation_enabled = True
augment_volume = 16


if augmentation_enabled ==  True:
  batch_size = int(batch_size/augment_volume)


start = time.time()

dataset_path = 'micro'
split_path = 'traintestlist'

depth, height, width = 32, 64, 64

# Define training set

train_dataset_vox = VoxelDataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    training=True,
)
train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,shuffle=True, num_workers=4)
#train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,sampler=BalancedBatchSampler(train_dataset_vox),shuffle=False, num_workers=4)


# Define test set
test_dataset_vox = VoxelDataset(
    dataset_path=dataset_path,
    split_path=split_path,
    split_number=split_number,
    training=False,
)
test_dataloader_vox = DataLoader(test_dataset_vox, batch_size=batch_size, shuffle=False, num_workers=4)

endtime = time.time()

print("Elapsed time : " + str(endtime-start))

gc.collect() 


# Network Model

In [ ]:
shutil.copyfile("/content/drive/My Drive/Alzheimer Competition/lr1e-4_dec8e-4_90.484_mcc_0.766.pth", "weight_3D.pth")
checkpoint_model = "weight_3D.pth"

# fold3_resnet101_aug16_32_64_64_acc_85.476_mcc_0.641.pth
# lr1e-4_dec1e-3_acc_89.649_mcc_0.744.pth
# lr1e-4_dec8e-4_90.484_mcc_0.766.pth

In [ ]:
import resnet
model = resnet.resnet101(
                num_classes=2,
                shortcut_type='B',
                sample_size=64,
                sample_duration=32)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

model.load_state_dict(torch.load(checkpoint_model))

### Model Hyperparameters

In [ ]:
#error = nn.BCEWithLogitsLoss().to(device)
error = nn.CrossEntropyLoss().to(device)

num_epochs = 50

learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-3)  #added L2 regularization

# Visualization and Saving Output

In [ ]:
def conf_mat(cf, epoch, acc, mcc, y_true, train_loss, train_acc, test_loss, test_acc):
  try:
    plt.imshow(cf,cmap=plt.cm.RdYlGn,interpolation='nearest')
    plt.colorbar()
    plt.title('Confusion Matrix without Normalization')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    tick_marks = np.arange(len(set(y_true))) # length of classes
    class_labels = ['Non Stalled','Stalled']
    tick_marks
    plt.xticks(tick_marks,class_labels)
    plt.yticks(tick_marks,class_labels)
    # plotting text value inside cells
    thresh = cf.max() / 2.
    for i,j in itertools.product(range(cf.shape[0]),range(cf.shape[1])):
        plt.text(j,i,format(cf[i,j],'d'),horizontalalignment='center',color='white' if cf[i,j] >thresh else 'black')
    #plt.show()
    os.makedirs("confusion_matrix", exist_ok=True)
    plt.savefig(f"confusion_matrix/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
    plt.close('all')

    os.makedirs("loss_acc_curve", exist_ok=True)

    ##creating subplot for loss,acc
    fig1, axs = plt.subplots(4,sharex=True,constrained_layout=True)
    axs[0].plot(train_loss, color = "red") 
    axs[0].set_title("Train loss")
    axs[1].plot(train_acc, color = "blue") 
    axs[1].set_title("Train Accuracy")
    axs[2].plot(test_loss, color = "green")
    axs[2].set_title("Test Loss")
    axs[3].plot(test_acc) 
    axs[3].set_title("Test Accuracy")
    #fig1.tight_layout()
    #plt.show()
    fig1.savefig(f"loss_acc_curve/epoch{epoch}_accuracy{round(acc.item(),3)}_mcc_{round(mcc.item(),3)}.png")
    plt.close(fig1)
  except:
    print("MCC is 0")

In [ ]:
#for cross entropy loss
def validation(epoch,train_loss, train_acc, test_loss, test_acc):

  correct = 0
  total = 0
  t_loss = 0
  y_true = []
  y_pred = []

  model.eval()

  for images, labels in test_dataloader_vox:

      y_true = np.append(y_true, labels.numpy())

      images = images.view(-1,3,depth,height,width)

      test = Variable(images.to(device), requires_grad=False)
      labels = Variable(labels.to(device), requires_grad=False)

      with torch.no_grad():
      
        # Forward propagation
        outputs = model(test)
        t_loss += error(outputs, labels)
        
        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())
        # Total number of labels
        total += len(labels)
        correct += (predicted == labels).sum()
  model.train()
  loss = t_loss.cpu().numpy() / float(total)
  test_loss.append(loss)
  accuracy = 100 * correct / float(total)
  test_acc.append(accuracy)

  mcc_score = mcc(y_true, y_pred)
  print(f"MCC score is {round(mcc_score,4)}")


  global MCC_SCORE
  if MCC_SCORE < mcc_score:
      MCC_SCORE = mcc_score
      os.makedirs("model_checkpoints", exist_ok=True)
      try:
        torch.save(model.state_dict(), f"model_checkpoints/3D_pointcloud_SimpleCNN_{epoch}for_cloud_size_32_64_64_acc_{round(accuracy.item(),3)}_mcc_{round(mcc_score.item(),3)}.pth")
      except:
        pass
  cf =confusion_matrix(y_true, y_pred)
  #print(cf)
  conf_mat(cf, epoch, accuracy, mcc_score, y_true, train_loss, train_acc, test_loss, test_acc)

  print('TESTING ---> Epoch: {} Loss: {} Accuracy: {} %'.format(epoch, round(loss,3), round(accuracy.item(),3)))


  return test_loss, test_acc


# Training

**TIP:** This training could take several hours depending on how many iterations you chose in the .cfg file. You will want to let this run as you sleep or go to work for the day, etc. However, Colab Cloud Service kicks you off it's VMs if you are idle for too long (30-90 mins).

To avoid this hold (CTRL + SHIFT + i) at the same time to open up the inspector view on your browser.

Paste the following code into your console window and hit **Enter**
```
function ClickConnect(){
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect,60000)
```

### Augmentation Code

In [ ]:
def augment(images, labels, augment_volume=2):

  [instances, channel, depth, height, width] = images.shape

  images_aug = torch.zeros((instances*augment_volume, channel, depth, height, width))
  labels_aug = torch.zeros(instances*augment_volume)

  for aug_type in range(augment_volume):
    augmented = im_aug(images, aug_type)
    
    images_aug[aug_type*instances:(aug_type+1)*instances, :, :, :, :] = augmented
    labels_aug[aug_type*instances:(aug_type+1)*instances] = labels

  return images_aug.float(), labels_aug.long()



def im_aug(images, aug_type):

    if aug_type == 0:
      return images

    elif aug_type == 1:
      return torch.rot90(images, 1, [3,4])

    elif aug_type == 2:
      return torch.rot90(images, 2, [3,4])

    elif aug_type == 3:
      return torch.rot90(images, 3, [3,4])

    elif aug_type == 4:
      temp = torch.flip(images, [2,3])
      return temp

    elif aug_type == 5:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 6:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 7:
      temp = torch.flip(images, [2,3])
      return torch.rot90(temp, 3, [3,4])

    elif aug_type == 8:
      temp = torch.transpose(images, 3, 4)
      return temp

    elif aug_type == 9:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 10:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 11:
      temp = torch.transpose(images, 3, 4)
      return torch.rot90(temp, 3, [3,4])

    elif aug_type == 12:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return temp

    elif aug_type == 13:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 1, [3,4])

    elif aug_type == 14:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 2, [3,4])

    elif aug_type == 15:
      temp = torch.flip(images, [2,3])
      temp = torch.transpose(temp, 3, 4)
      return torch.rot90(temp, 3, [3,4])


# Train

In [ ]:
### TRAINING code

MCC_SCORE = -1


train_loss = []   #to keep track of loss with respect to number of epoch 
test_loss = []
iteration_list = []
train_acc = []
test_acc = []

for epoch in tqdm(range(num_epochs)):
    #count = 1
    accuracy_list = []
    loss_list = []
    for i, (images, labels) in enumerate(train_dataloader_vox):
        correct = 0
        
        if augmentation_enabled == True:
          images, labels = augment(images, labels, augment_volume)

        images = images.view(-1,3,depth,height,width)


        #train = Variable(images)    #to use on cpu
        #labels = Variable(labels)    #to use on cpu

        train_img = Variable(images.to(device), requires_grad=True)
        labels = Variable(labels.to(device), requires_grad=False)

        # Clear gradients
        optimizer.zero_grad()
        # Forward propagation
        outputs = model(train_img)
        # Calculate softmax and ross entropy loss
        loss = error(outputs, labels)
        
        #labels = labels.view(-1, 1)  #only for BCELogitsloss
        #loss = error(outputs.float(), labels.float())  #if BCELoss is measured
        
        # Calculating gradients
        loss.backward()
        # Update parameters
        optimizer.step()
        
        predicted = torch.max(outputs.data, 1)[1]   
        correct = (predicted == labels).sum()
        total = len(labels)        
        accuracy = 100 * correct / float(total)

        '''
        ###For BCELoss
        predicted = torch.tensor([0 if i<=0.5 else 1 for i in outputs]).to(device)
        accuracy = 100 * (predicted.detach() == labels).cpu().numpy().mean()
        '''
        #accuracy_list = np.append(accuracy_list, accuracy)

        accuracy_list = np.append(accuracy_list, accuracy.cpu().numpy())
        loss_list = np.append(loss_list, loss.detach().cpu().numpy())
        #print(f"Training Mini Batch--> Epoch:{epoch} Iteration:{count} Loss :{loss} Accuracy: {accuracy}")
        #count += 1
    final_loss = np.mean(loss_list)
    final_acc = np.mean(accuracy_list)
    print(f"TRAINING ---> Epoch: {epoch} Loss: {round(final_loss,4)} Accuracy: {round(final_acc,4)}")
    train_loss.append(final_loss)
    train_acc.append(final_acc)
    gc.collect() 
    test_loss , test_acc = validation(epoch, train_loss, train_acc, test_loss, test_acc)
    
    train_dataset_vox = VoxelDataset(
        dataset_path=dataset_path,
        split_path=split_path,
        split_number=split_number,
        training=True,
    )
    train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,shuffle=True, num_workers=4)
    #train_dataloader_vox = DataLoader(train_dataset_vox, batch_size= batch_size,sampler=BalancedBatchSampler(train_dataset_vox),shuffle=False, num_workers=4)


# Testing Validation Augmentation

In [ ]:
def validation_aug(augment_enabled=False, augment_volume=16):

  correct = 0
  total = 0
  t_loss = 0
  y_true = []
  y_pred = []

  model.eval()

  for images, labels in test_dataloader_vox:

      y_true = np.append(y_true, labels.numpy())

      with torch.no_grad():

        image_s = images.view(-1,3,depth,height,width)
        test = Variable(image_s.to(device), requires_grad=False)
        labels = Variable(labels.to(device), requires_grad=False)

        # Forward propagation
        outputs = model(test)

        if (augment_enabled == True):

            for aug_type in range(1, augment_volume):

              image_s = im_aug(images, aug_type)
              image_s = image_s.view(-1,3,depth,height,width)
              test = Variable(image_s.to(device), requires_grad=False)
              outputs = outputs + model(test)

            outputs = outputs / augment_volume

        t_loss += error(outputs, labels)
        
        # Get predictions from the maximum value
        predicted = torch.max(outputs.data, 1)[1]
        #print(f"prediction size are {predicted.shape}")
        y_pred = np.append(y_pred, predicted.cpu().numpy())
        # Total number of labels
        total += len(labels)
        correct += (predicted == labels).sum()


  model.train()
  loss = t_loss.cpu().numpy() / float(total)
  accuracy = 100 * correct / float(total)

  mcc_score = mcc(y_true, y_pred)
  print(f"MCC score is {round(mcc_score,4)}")

  print('TESTING ---> Loss: {} Accuracy: {} %'.format(round(loss,3), round(accuracy.item(),3)))


In [ ]:
validation_aug(augment_enabled=False, augment_volume=augment_volume) 
validation_aug(augment_enabled=True, augment_volume=4) 
validation_aug(augment_enabled=True, augment_volume=8) 
validation_aug(augment_enabled=True, augment_volume=16) 

print("Done")
